In [25]:
# HIDDEN
import sys
sys.path.insert(0, "/Users/blankjul/workspace/pyrecorder")

from IPython.display import display, HTML, Image
from base64 import b64encode


def display(fname):
    
    extension = fname.split(".")[-1]
    
    if  extension == "gif":
        gif = open(fname,'rb').read()  
        return Image(b64encode(gif),embed=True)
    
    elif extension == "mp4" or extension == "webm":
        from IPython.display import HTML, display
        
        video = open(fname,'rb').read()   
        data = "data:video/mp4;base64," + b64encode(video).decode()
        
        html = """
        <div align="middle">
            <video controls>
            <source src="%s" type="video/%s">
        </video>
        </div>
        """
        
        return display(HTML(data=html % (data, extension)))

    elif fname.endswith("html"):
        
        return HTML(filename=fname)


# pyrecorder

![logo](_static/pyrecorder.png)

**Github:** https://github.com/anyoptimization/pyrecorder

## Installation

The framework is available at the PyPi Repository:

## Matplotlib

Please note that the example below are using the `vp80` codec to create a video which can be played in a browser and also this documentation. Nevertheless, without specifing the codec `mp4v` is used by default.

### Video

In [26]:
import numpy as np
import matplotlib.pyplot as plt

from pyrecorder.recorder import Recorder
from pyrecorder.writers.video import Video
from pyrecorder.converters.matplotlib import Matplotlib

# create a writer that takes the 
writer = Video("example.webm", codec='vp80')

# use the with statement to close the recorder when done
with Recorder(writer) as rec:
    
    # use black background for this plot
    plt.style.use('dark_background')

    # record 10 different snapshots
    for t in range(50, 500, 5):

        a = np.arange(t) * 0.1
        plt.plot(a * np.sin(a), a * np.cos(a))
        plt.xlim(-50, 50)
        plt.ylim(-50, 50)
        plt.axis('off')

        # use the record to store the current plot
        rec.record()
        
# revert to default settings for other plots
plt.style.use('default')

When the code has finished, the video has been written to the specified filename `example.mp4`. Let us look what has been recorded:

In [27]:
display("example.webm")

For this example the default settings have been used and the global drawing space of Matplotlib is recorded. Let us look at another example with a few modifications:

In [28]:
import numpy as np
import matplotlib.pyplot as plt

from pyrecorder.recorder import Recorder
from pyrecorder.writers.video import Video
from pyrecorder.converters.matplotlib import Matplotlib

# initialize the converter which is creates an image when `record()` is called
converter = Matplotlib(dpi=120)

writer = Video("example2.webm", codec='vp80')

rec = Recorder(writer, converter=converter)

for t in range(10):

    # let us create a local figure object with two sub figures
    fig, (ax1, ax2) = plt.subplots(2, figsize=(3, 4))

    X = np.random.random((100, 2))
    ax1.scatter(X[:, 0], X[:, 1], color="green")

    X = np.random.random((100, 2))
    ax2.scatter(X[:, 0], X[:, 1], color="red")
    
    # fix the size of figure and legends
    fig.tight_layout()

    # take a snapshot the specific figure object with the recorder
    rec.record(fig=fig)
    
    
rec.close()

In [29]:
display("example2.webm")

### GIF

In [30]:
import matplotlib.pyplot as plt
import numpy as np

from pyrecorder.recorder import Recorder
from pyrecorder.writers.gif import GIF

with Recorder(GIF("example.gif", duration=0.2)) as rec:

    for t in range(0, 200, 5):
        
        x = np.linspace(0, 4, 100)
        y = np.sin(2 * np.pi * (x - 0.01 * t))
        plt.plot(x, y)

        rec.record()

![My GIF](example.gif)

## Contact